In [ ]:
from utils import (
    # for adding model prediction results to csv file
    add_yolov5_conf_scores,
    add_efficientnet_conf_scores,
    add_detr_conf_scores,
)

import pandas as pd
import numpy as np

## Params

In [ ]:
gov_csv_dir = "data/gov_data_for_downloading_sp_06.csv"

model_1_name = "efficientnet" # classification model
model_2_name = "yolov5" # object detection model
model_3_name = "detr"   # object detection model
model_1_pred_folder = f"data/model_predictions/{model_1_name}_predictions" # contains model 1 prediction results
model_2_pred_folder = f"data/model_predictions/{model_2_name}_predictions" # contains model 2 prediction results
model_3_pred_folder = f"data/model_predictions/{model_3_name}_predictions" # contains model 3 prediction results

# column names - no need to modify these variables
tile_id_col = "point_id" # for section 4

## 1. Add model prediction scores to DataFrame

In [ ]:
# read csv files
df = pd.read_csv(gov_csv_dir)

# adding model prediction results to DataFrame

model_1_conf_col = f"conf_{model_1_name}" # "conf_efficientnet"
model_2_conf_col = f"conf_{model_2_name}" # "conf_yolov5"
model_3_conf_col = f"conf_{model_3_name}" # "conf_detr"

# put NaN values in three model conf score columns
df[model_1_conf_col] = np.nan
df[model_2_conf_col] = np.nan
df[model_3_conf_col] = np.nan


#####  add model probability scores to DataFrame  #####

# add model 1 (EfficientNet) probability scores to DataFrame
df = add_efficientnet_conf_scores(original_df = df,
                                  predictions_folder = model_1_pred_folder,
                                  school_class_id = 1, 
                                  school_conf_col = model_1_conf_col, 
                                  image_id_col = tile_id_col)

# add model 2 (YOLOv5) probability scores to DataFrame
df = add_yolov5_conf_scores(original_df = df, 
                            predictions_folder = model_2_pred_folder,
                            school_conf_col = model_2_conf_col, 
                            image_id_col = tile_id_col)

# add model 3 (DeTR) probability scores to DataFrame
df = add_detr_conf_scores(original_df = df, 
                          predictions_folder = model_3_pred_folder,
                          school_conf_col = model_3_conf_col, 
                          image_id_col = tile_id_col)

# replace NaNs with zeros
df[model_1_conf_col] = df[model_1_conf_col].fillna(0)
df[model_2_conf_col] = df[model_2_conf_col].fillna(0)
df[model_3_conf_col] = df[model_3_conf_col].fillna(0)

# reset index
df = df.reset_index(drop = True)

In [ ]:
"""  Save csv file  """

df.to_csv("data/mongolia_gov_data_model_conf_scores_added.csv", index = False)